# Operations 

## Sections
1.  UFuncs: Index Alignment in Series
2.  Index alignment in DataFrame
3.  Handling Missing Data
4.  Read and write CSV and XLS files

<a id = 'functions'/>

## 1. UFuncs: Index Alignment in Series
For binary operations on two Series or DataFrame objects, Pandas will align indices
in the process of performing the operation. This is very convenient when you are
working with incomplete data, as we’ll see in some of the examples

In [3]:
import pandas as pd
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
'New York': 19651127}, name='population')

In [4]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

The resulting array contains the union of indices of the two input arrays, which we
could determine using standard Python set arithmetic on these indices

In [5]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

This index matching is implemented this way for any of Python’s built-in arithmetic expressions; any missing values
are filled in with NaN by default

In [6]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

If using NaN values is not the desired behavior, we can modify the fill value using
appropriate object methods in place of the operators. For example, calling A.add(B)
is equivalent to calling A + B, but allows optional explicit specification of the fill value
for any elements in A or B that might be missing

In [7]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

<a id = 'index'/>

## 2. Index alignment in DataFrame

A similar type of alignment takes place for both columns and indices when you are
performing operations on DataFrames

In [8]:
import numpy as np
A = pd.DataFrame(np.random.randint(0, 20, (2, 2)), columns=list('AB'))
A

,A,B
0,6,0
1,3,19


In [9]:
B = pd.DataFrame(np.random.randint(0, 10, (3, 3)), columns=list('BAC'))
B

,B,A,C
0,2,0,9
1,5,6,6
2,1,6,1


In [10]:
A + B

,A,B,C
0,6.0,2.0,NaN
1,9.0,24.0,NaN
2,NaN,NaN,NaN


 <a id = 'missing'/>

## 3. Handling Missing Data

* The difference between data found in many tutorials and data in the real world is that real-world data is rarely clean and homogeneous. In particular, many interestingdatasets will have some amount of data missing. To make matters even more complicated, different data sources may indicate missing data in different ways.

* In this section, we will discuss some general considerations for missing data, discuss how Pandas chooses to represent it, and demonstrate some built-in Pandas tools for handling missing data in Python. Here and throughout the book, we’ll refer to missing data in general as null, NaN, or NA values.

Generally, they revolve around one of two strategies: using a
mask that globally indicates missing values, or choosing a sentinel value that indicates
a missing entry.

1. Sentinel Value [-9999, NaN]
2. Mask

In [14]:
# None: Python Missing Data
import numpy as np
import pandas as pd
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

This dtype=object means that the best common type representation NumPy could
infer for the contents of the array is that they are Python objects. While this kind of
object array is useful for some purposes, any operations on the data will be done at
the Python level, with much more overhead than the typically fast operations seen for
arrays with native types

In [15]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
284 ms ± 67.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

dtype = int
4.75 ms ± 440 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



In [17]:
"""The use of Python objects in an array also means that if you perform aggregations
like sum() or min() across an array with a None value, you will generally get an error"""

vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [20]:
# NaN missing data
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

In [21]:
vals2.sum()

nan

In [22]:
np.nansum(vals2)

8.0

### 3.1 NaN and None in Pandas

NaN and None both have their place, and Pandas is built to handle the two of them
nearly interchangeably, converting between them where appropriate

In [23]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

### 3.2 Operating on Null Values

In [24]:
# Detecting null values 
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [25]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [26]:
data[data.notnull()]

0        1
2    hello
dtype: object

In [27]:
# Dropping null values
data.dropna()

0        1
2    hello
dtype: object

In [28]:
df = pd.DataFrame([[1, np.nan, 2],
[2, 3, 5],
[np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


We cannot drop single values from a DataFrame; we can only drop full rows or full
columns. Depending on the application, you might want one or the other, so
dropna() gives a number of options for a DataFrame

In [29]:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [30]:
df.dropna(axis=1)

,2
0,2
1,5
2,6


In [31]:
df[3] = np.nan
print(df)

     0    1  2   3
0  1.0  NaN  2 NaN
1  2.0  3.0  5 NaN
2  NaN  4.0  6 NaN


In [32]:
df.dropna(axis='columns', thresh=3) #threshold

,2
0,2
1,5
2,6


In [33]:
df.dropna(axis='columns', how='all') #how attribute

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


### 3.3 Filling Null Values
Sometimes rather than dropping NA values, you’d rather replace them with a valid
value. This value might be a single number like zero, or it might be some sort of
imputation or interpolation from the good values. You could do this in-place using
the isnull() method as a mask, but because it is such a common operation Pandas
provides the fillna() method, which returns a copy of the array with the null values
replaced.

In [34]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [35]:
data.fillna(0) # a single value such as zero

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [36]:
data.fillna(method = 'ffill') # forward-fill

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [37]:
data.fillna(method = 'bfill') # backward-fill

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

For DataFrames, the options are similar, but we can also specify an axis along which
the fills take place

In [38]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [39]:
df.fillna(method = 'ffill', axis =1 )

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


<a id = 'read'/>

## 4. Read and write CSV and XLS files

In [40]:
import pandas as pd
df = pd.read_csv('weather_data.csv')
df

,day,temperature,windspeed,event
0,1/1/2017,32,6,Rain
1,1/2/2017,35,7,Sunny
2,1/3/2017,28,2,Snow
3,1/4/2017,24,7,Snow
4,1/5/2017,32,4,Rain
5,1/6/2017,31,2,Sunny


In [41]:
#INSTALL: pip3 install xlrd

#read excel file 
df = pd.read_excel('weather_data.xlsx')
df

,day,temperature,windspeed,event
0,1/1/2017,32,6,Rain
1,1/2/2017,35,7,Sunny
2,1/3/2017,28,2,Snow
3,1/4/2017,24,7,Snow
4,1/5/2017,32,4,Rain
5,1/6/2017,31,2,Sunny


In [42]:
#write DF to csv
df.to_csv('new.csv') 
df.to_csv('new_noIndex.csv', index=False)

In [43]:
# INSTALL: pip3 install openpyxl

#write DF to Excel
df.to_excel('new.xlsx', sheet_name='weather_data')

### 4.1 GROUP-BY

In [44]:
import pandas as pd
df = pd.read_csv('weather_data_cities.csv')
df #weather by cities

,day,city,temperature,windspeed,event
0,1/1/2017,new york,32,6,Rain
1,1/2/2017,new york,36,7,Sunny
2,1/3/2017,new york,28,12,Snow
3,1/4/2017,new york,33,7,Sunny
4,1/1/2017,mumbai,90,5,Sunny
5,1/2/2017,mumbai,85,12,Fog
6,1/3/2017,mumbai,87,15,Fog
7,1/4/2017,mumbai,92,5,Rain
8,1/1/2017,paris,45,20,Sunny
9,1/2/2017,paris,50,13,Cloudy


In [45]:
g = df.groupby('city')
g

In [46]:
for city, city_df in g:
    print(city)
    print(city_df)

mumbai
        day    city  temperature  windspeed  event
4  1/1/2017  mumbai           90          5  Sunny
5  1/2/2017  mumbai           85         12    Fog
6  1/3/2017  mumbai           87         15    Fog
7  1/4/2017  mumbai           92          5   Rain
new york
        day      city  temperature  windspeed  event
0  1/1/2017  new york           32          6   Rain
1  1/2/2017  new york           36          7  Sunny
2  1/3/2017  new york           28         12   Snow
3  1/4/2017  new york           33          7  Sunny
paris
         day   city  temperature  windspeed   event
8   1/1/2017  paris           45         20   Sunny
9   1/2/2017  paris           50         13  Cloudy
10  1/3/2017  paris           54          8  Cloudy
11  1/4/2017  paris           42         10  Cloudy


In [47]:
#or to get specific group
g.get_group('new york')


,day,city,temperature,windspeed,event
0,1/1/2017,new york,32,6,Rain
1,1/2/2017,new york,36,7,Sunny
2,1/3/2017,new york,28,12,Snow
3,1/4/2017,new york,33,7,Sunny


In [48]:
#Find maximum temperature in each of the cities
print(g.max())

               day  temperature  windspeed  event
city                                             
mumbai    1/4/2017           92         15  Sunny
new york  1/4/2017           36         12  Sunny
paris     1/4/2017           54         20  Sunny


In [49]:
print(g.mean())


          temperature  windspeed
city                            
mumbai          88.50       9.25
new york        32.25       8.00
paris           47.75      12.75


In [50]:
print(g.describe())

         temperature                                                   \
               count   mean       std   min    25%   50%    75%   max   
city                                                                    
mumbai           4.0  88.50  3.109126  85.0  86.50  88.5  90.50  92.0   
new york         4.0  32.25  3.304038  28.0  31.00  32.5  33.75  36.0   
paris            4.0  47.75  5.315073  42.0  44.25  47.5  51.00  54.0   

         windspeed                                                 
             count   mean       std  min   25%   50%    75%   max  
city                                                               
mumbai         4.0   9.25  5.057997  5.0  5.00   8.5  12.75  15.0  
new york       4.0   8.00  2.708013  6.0  6.75   7.0   8.25  12.0  
paris          4.0  12.75  5.251984  8.0  9.50  11.5  14.75  20.0  


### 4.2 concatenate Data Frames

In [51]:
import pandas as pd
india_weather = pd.DataFrame({
    "city": ["mumbai","delhi","banglore"],
    "temperature": [32,45,30],
    "humidity": [80, 60, 78]
})

india_weather

,city,temperature,humidity
0,mumbai,32,80
1,delhi,45,60
2,banglore,30,78


In [52]:
us_weather = pd.DataFrame({
    "city": ["new york","chicago","orlando"],
    "temperature": [21,14,35],
    "humidity": [68, 65, 75]
})
us_weather

,city,temperature,humidity
0,new york,21,68
1,chicago,14,65
2,orlando,35,75


In [53]:
#concate two dataframes
df = pd.concat([india_weather, us_weather])
df

,city,temperature,humidity
0,mumbai,32,80
1,delhi,45,60
2,banglore,30,78
0,new york,21,68
1,chicago,14,65
2,orlando,35,75


In [54]:
#if you want continuous index
df = pd.concat([india_weather, us_weather], ignore_index=True)
df

,city,temperature,humidity
0,mumbai,32,80
1,delhi,45,60
2,banglore,30,78
3,new york,21,68
4,chicago,14,65
5,orlando,35,75


In [55]:
df = pd.concat([india_weather, us_weather],axis=1)
df

,city,temperature,humidity,city,temperature,humidity
0,mumbai,32,80,new york,21,68
1,delhi,45,60,chicago,14,65
2,banglore,30,78,orlando,35,75


### 4.3 Merge DataFrames

In [56]:
temperature_df = pd.DataFrame({
    "city": ["mumbai","delhi","banglore", 'hyderabad'],
    "temperature": [32,45,30,40]})
temperature_df

,city,temperature
0,mumbai,32
1,delhi,45
2,banglore,30
3,hyderabad,40


In [57]:
humidity_df = pd.DataFrame({
    "city": ["delhi","mumbai","banglore"],
    "humidity": [68, 65, 75]})
humidity_df

,city,humidity
0,delhi,68
1,mumbai,65
2,banglore,75


In [58]:
#merge two dataframes with out explicitly mention index
df = pd.merge(temperature_df, humidity_df, on='city')
df

,city,temperature,humidity
0,mumbai,32,65
1,delhi,45,68
2,banglore,30,75


In [59]:
#OUTER-JOIN
df = pd.merge(temperature_df, humidity_df, on='city', how='outer')
df

,city,temperature,humidity
0,mumbai,32,65.0
1,delhi,45,68.0
2,banglore,30,75.0
3,hyderabad,40,NaN
